In [149]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [112]:
origX = pd.read_csv("../Datasets/train_x.csv", sep=",", header=None).as_matrix().reshape(-1,64,64)
origY = pd.read_csv("../Datasets/train_y.csv", sep=",", header=None).as_matrix() 

In [113]:
#origX = origX.reshape(-1, 4096) # reshape 
#origY = origY.reshape(-1)
origX.shape

(50000, 64, 64)

# Preprocessing

In [114]:
def binary_tresholding(image):
    for i in range(len(image)):
        for j in range(len(image)):
            if image[i, j] < 254:
                image[i, j] = 0
    return image

In [115]:
#Binarize all train images
images_train = np.zeros(origX.shape)
for i in range(origX.shape[0]):
    images_train[i] = binary_tresholding(origX[i])
    
#Binarize all test images
images_test = np.zeros(origTestX.shape)
for i in range(origTestX.shape[0]):
    images_test[i] = binary_tresholding(origTestX[i])

In [116]:
def find_the_largest_digit_center(image):
    thresh = binary_tresholding(image)
    thresh = thresh.astype('uint8')

    im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    Ws = [] 
    Hs = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        Ws.append(w)
        Hs.append(h)
    max_h = max(Hs)
    max_w = max(Ws)
    max_l = max(max_h,max_w)

    # Find the index of the contour with largest width
    if max_l == max_h:
        max_index = np.argmax(Hs)
    else:
        max_index = np.argmax(Ws)
    x,y,w,h = cv2.boundingRect(contours[max_index])

    #to create a filter to remove all other digits and keep just the biggest one
    filter_ = np.zeros(image.shape)
    filter_ = cv2.rectangle(filter_,(x,y),(x+max_l,y+max_l),(255,255,255),-1)
    
    #find center of the component
    px = x+max_l/2
    py = y+max_l/2

    preprocessed_img = np.zeros(origX[0].shape)
    preprocessed_img = thresh*filter_

    # To shift the largest component to the center of an image
    preprocessed_img = preprocessed_img.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(preprocessed_img, connectivity=4)
    sizes = stats[:, -1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
        
    img2 = np.zeros(output.shape)
    img2[output == max_label] = 255.
    
    # To shift the largest component to the center of an image
    px = 32 - centroids[max_label][0]
    py = 32 - centroids[max_label][1]
    M = np.float32([[1,0,px],[0,1,py]])
    dst = cv2.warpAffine(img2,M,(64,64))
    
    return dst[18:46,18:46]

In [117]:
# find the biggest component in all dataset and shift it to center
prepTrainX = np.zeros([50000, 784])
for i in range(len(origX)):
    img1 = find_the_largest_digit_center(images_train[i, :])
    img1 = img1.reshape((1, 784))
    prepTrainX[i, :] = img1
    
prepTestX = np.zeros([10000, 784])
for i in range(len(origTestX)):
    img2 = find_the_largest_digit_center(images_test[i, :])
    img2 = img2.reshape((1, 784))
    prepTestX[i, :] = img2

In [ ]:
# takes a numpy 1-dimensional array and writes each value in a different row on the file
def write_solution(filename, y_pred):
    file = open(filename,"w")
    file.write("Id,Label" + "\n")
    for i in range(y_pred.shape[0]):
        file.write(repr(i) + "," + repr(int(y_pred[i])) + "\n")

In [ ]:
prepTrainX.shape

# Neural Network

In [153]:
# For cross validation parameter tweaking
x_train1 = prepTrainX[10000:]
y_train1 = origY[10000:]
x_valid1 = prepTrainX[:10000]
y_valid1 = origY[:10000]

x_train2 = np.concatenate((prepTrainX[:10000], prepTrainX[20000:]))
y_train2 = np.concatenate((origY[:10000], origY[20000:]))
x_valid2 = prepTrainX[10000:20000]
y_valid2 = origY[10000:20000]

x_train3 = np.concatenate((prepTrainX[:20000], prepTrainX[30000:]))
y_train3 = np.concatenate((origY[:20000], origY[30000:]))
x_valid3 = prepTrainX[20000:30000]
y_valid3 = origY[20000:30000]

x_train4 = np.concatenate((prepTrainX[:30000], prepTrainX[40000:]))
y_train4 = np.concatenate((origY[:30000], origY[40000:]))
x_valid4 = prepTrainX[30000:40000]
y_valid4 = origY[30000:40000]

x_train5 = prepTrainX[:40000]
y_train5 = origY[:40000]
x_valid5 = prepTrainX[40000:]
y_valid5 = origY[40000:]

In [154]:
x = np.divide(x_train1,255.0) # normalize pixels
x_v = np.divide(x_valid1,255.0) # normalize pixels
y = y_train1
y_v = y_valid1

In [ ]:
# Parameters to tweak
batchSize = 1
epochs = 100
learning_rate = .005
numHiddenLayers = 6
hiddenLayerNodes = [2, 4, 7, 3, 8, 5]
if len(hiddenLayerNodes) != numHiddenLayers:
    print("PROBLEM IN INPUT!!!")

In [ ]:
# Initializing weights and bias
weights = []
biases = []
for i in range(numHiddenLayers+1):
    if i == 0:
        w = np.random.randn(784, hiddenLayerNodes[i])
        b = np.random.randn(1, hiddenLayerNodes[i])
    elif i != numHiddenLayers:
        w = np.random.randn(hiddenLayerNodes[i-1], hiddenLayerNodes[i])
        b = np.random.randn(1, hiddenLayerNodes[i])
    else:
        w = np.random.randn(hiddenLayerNodes[i-1], 10) # 10 output nodes
        b = np.random.randn(1, 10)
    weights.append(w)
    biases.append(b)

In [ ]:
batches = [x[k: k + batchSize] for k in range(0, len(x), batchSize)]
yBatches = [y[k: k + batchSize] for k in range(0, len(y), batchSize)]

In [ ]:
def sigmoid(z, deriv):
    sig = 1.0 / (1.0 + np.exp(-z))
    if deriv:
        return sig * (1 - sig)
    else:
        return sig

In [ ]:
from copy import deepcopy
# NN algorithm

for ep in range(epochs):
    for i in range(len(batches)):
        xb = batches[i]
        yb = yBatches[i]
        
        currentBatchSize = len(xb)

        # feed forward
        a = xb
        As = [a] # activations (right side of node)
        Zs = [] # (left side of node)
        for l in range(len(weights)): # for x total layers (including input and output) this will iterate x-1 times
            z = np.dot(a, weights[l]) + biases[l]
            a = sigmoid(z, False)
            Zs.append(z)
            As.append(a)

        # Back propagation

        e = np.zeros((currentBatchSize, 10)) # Derivatives of quadratic deviation
        for j in range(len(As[-1])): # j represents image index in batch
            e[j] = deepcopy(As[-1][j]) # delta = prediction - 0 assuming label is 0, deepcopy to pass by value instead of reference
            e[j, int(yb[j])] -= 1 # delta = prediction - 1 because label is 1 at index yb

        deltas = [] # list of delta layers
        # delta[0] represents the first layer (starting from right) of deltas for all images in batch
        # delta[0][0, :] represents the first layer (starting from right) of deltas for the first image
        # delta[0][0, 0] represents the delta of the first node of the first (starting from right) layer of deltas for the first image

        #Calculate deltas in output layer                
        D = [np.diag(sigmoid(Zs[-1][j], True)) for j in range(currentBatchSize)] # Derivatives stored in output layer
        d = np.array([np.dot(D[j], e[j]) for j in range(currentBatchSize)]) # output layer deltas
        deltas.append(d)

        bGradients = [np.zeros((currentBatchSize, b.shape[1])) for b in biases]
        bGradients[-1] = d
        wGradients = [np.zeros(w.shape) for w in weights]
        wGradients[-1] = -learning_rate * np.dot(As[-2].T, d)

        # calculating deltas
        for k in range(len(weights)-1, 0, -1):            
            D = [np.diag(sigmoid(Zs[k-1][j], True)) for j in range(currentBatchSize)] # Derivatives stored in next hidden layer
            d = np.array([np.dot(np.dot(D[j], weights[k]), d[j]) for j in range(currentBatchSize)]) # next hidden layer deltas
            bGradients[k-1] = d
            wGradients[k-1] = -learning_rate * np.dot(As[k-1].T, d)
            deltas.append(d)

        # bGradients[0] represents first layer
        # bGradients[0][i, j] represents for first layer, mini batch i and bias value j

        # wGradients[0] represents first layer
        # wGradients[0][i, j] represents for first layer sum of weight edges from node i to j for all mini batches (images) in batch

        # divide gradients for avg
        wGradientsAvg = [np.zeros(w.shape) for w in weights]
        for l in range(len(wGradients)):
            wGradientsAvg[l] = wGradients[l] / currentBatchSize

        bGradientsAvg = [np.zeros(b.shape) for b in biases]
        for l in range(len(bGradients)):
            bGradientsAvg[l][0,:] = sum(bGradients[l])
            bGradientsAvg[l] /= currentBatchSize

        # update weights
        for l in range(len(weights)):
            weights[l] += wGradientsAvg[l]
            biases[l] += bGradientsAvg[l]

In [ ]:
# Evaluating training data

# Last feed forward for evaluation
a = x
for l in range(len(weights)): # for x total layers (including input and output) this will iterate x-1 times
    z = np.dot(a, weights[l]) + biases[l]
    a = sigmoid(z, False)

values = [np.argmax(a_i) for a_i in a]
binaryResults = []
for i in range(len(values)):
    binaryResults.append(int(values[i] == y[i]))
    
score = sum(binaryResults) / len(values)

In [ ]:
print(values[:100])
print(np.unique(values))
print(score)

In [ ]:
# Evaluating validation data

# Last feed forward for evaluation
a = x_v
for l in range(len(weights)): # for x total layers (including input and output) this will iterate x-1 times
    z = np.dot(a, weights[l]) + biases[l]
    a = sigmoid(z, False)

values = [np.argmax(a_i) for a_i in a]
binaryResults = []
for i in range(len(values)):
    binaryResults.append(int(values[i] == y_v[i]))
    
score = sum(binaryResults) / len(values)

In [ ]:
print(values[:100])
print(np.unique(values))
print(score)